In [13]:
import torch
import numpy as np
import torch.nn as nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from mlp import MLP

In [14]:
X = np.load('./data/lbp/features.npy').astype(np.float32) # [num, 256]
y = np.load('./data/lbp/labels.npy').astype(np.int64) # [num, 4]

device = 'cpu'
# device = 'mps'
print("Device is set to:", device)

# Example data (replace with your actual data)
# X = torch.tensor(features, dtype=torch.float, device=device)
# y = torch.tensor(labels, dtype=torch.float, device=device)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# train_dataset = TensorDataset(X_train, y_train)
# val_dataset = TensorDataset(X_val, y_val)

Device is set to: cpu


In [15]:
from skorch.callbacks import GradientNormClipping

net = NeuralNetClassifier(
    MLP,
    module__input_size=256,
    module__output_size=4,
    max_epochs=200,
    lr=0.01,
    iterator_train__shuffle=True,
    callbacks=[GradientNormClipping(5.0)],  # Clip gradients at 5.0
)

In [16]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

# Check for NaNs and infinite values in data
print(np.isnan(X_train).sum(), np.isnan(y_train).sum())
print(np.isinf(X_train).sum(), np.isinf(y_train).sum())

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Ensure labels are in integer format
y_train = y_train.argmax(axis=1) if len(y_train.shape) > 1 and y_train.shape[1] > 1 else y_train
y_val = y_val.argmax(axis=1) if len(y_val.shape) > 1 and y_val.shape[1] > 1 else y_val

param_grid = {
    'module__hidden_sizes': [[128], [256], [256, 128], [128, 64], [256, 128, 64], [128, 64, 32]],
    'module__dropout_rate': [0.2, 0.3, 0.5],
    'lr': [0.001, 0.01],
    'batch_size': [32, 64]
}

cv = StratifiedKFold(n_splits=3)

0 0
0 0


In [ ]:
gs = GridSearchCV(net, param_grid, refit=True, cv=cv, scoring='accuracy', verbose=2)

gs.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.4823       -0.0621  0.1414
      2       -0.3050       0.4823       -0.5210  0.1284
      3       -0.6675       0.4823       -0.8151  0.1225
      4       -0.9260       0.4823       -1.0366  0.1266
      5       -1.1246       0.4823       -1.2160  0.1358
      6       -1.2885       0.4823       -1.3672  0.1325
      7       -1.4294       0.4823       -1.4983  0.1441
      8       -1.5513       0.4823       -1.6141  0.1290
      9       -1.6611       0.4823       -1.7179  0.1301
     10       -1.7611       0.4823       -1.8119  0.1250
     11       -1.8532       0.4823       -1.8978  0.1298
     12       -1.9308       0.4823       -1.9770  0.1268
     13       -2.0104       0.4823       -2.0505  0.1285
     14       -2.0807       0.4823       -2.1189  0.1258
     15       -2.1445     

In [ ]:
print(f'Best Params: {gs.best_params_}')
print(f'Best Cross-Validation Accuracy: {gs.best_score_}')

# Evaluate on validation set
y_pred = gs.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')